In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge, LogisticRegression, LogisticRegressionCV as Logit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, make_scorer
from sklearn.decomposition import NMF
import nltk
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
with open('data/fires_df.p', 'rb') as file:
    fires_df = pickle.load(file)

In [2]:
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [3]:
with open('data/predictor.p', 'rb') as file:
    predictor = pickle.load(file)

In [347]:
fires_df = pd.read_csv('data/Fire_Incidents.csv')

C:\Users\pengj\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (11,12,19,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


15_day_hold?
application_number
assessor_stories
avs_street_name
avs_street_sfx
block
city
company_name
contact_name
contact_phone
contractorphone
description
estimated_cost
existing_stories
existing_units
existing_use
expiration_date
file_date
first_name
form_number
last_name
lot
no of pages
plansets
proposed_stories
proposed_units
proposed_use
revised_cost
state
status
status_code
status_date
street
street_number
street_number.1
street_number_sfx
street_suffix
unit
unit_sfx
voluntary soft-story retrofit
zip_code
address

In [4]:
def data_processing(predictor):
    pred = predictor.copy()
    pred = pred[pred['status'] == 'ISSUED'].reset_index().drop('index', axis = 1)
    return pred

In [5]:
def data_dropping(predictor):
    pred = predictor.copy()
    drop = str.split("application_number first_name form_number last_name lot status_code street street_suffix unit unit_sfx zip_code address street_number street_number.1 street_number_sfx")
    return pred.drop(drop, axis = 1)

In [6]:
pred = data_processing(data_dropping(predictor))

In [377]:
pred['existing_stories'].describe()

count    242870.000000
mean          6.179833
std           9.503065
min           0.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          67.000000
Name: existing_stories, dtype: float64

In [388]:
pred['has_existing_stories'] = ~pred['existing_stories'].isnull()

# Testing: just use description

In [419]:
sample_idx_2 = np.random.choice(len(pred), 1500)

In [8]:
sample_idx = np.random.choice(len(pred), 1000)

In [519]:
sample = pred.iloc[sample_idx_2]

In [520]:
docs_sample = sample[['description', 'proposed_use', 'existing_use']].fillna('').apply(lambda x: ' '.join(x), axis = 1)

In [521]:
docs_sample = docs_sample.fillna('')

In [522]:
target_sample = target.iloc[sample_idx_2]

In [438]:
vec = TfidfVectorizer(stop_words = 'english', max_features = 6000)

In [439]:
vec.vocabulary_;

AttributeError: 'TfidfVectorizer' object has no attribute 'vocabulary_'

In [523]:
X = vec.fit_transform(docs_sample).toarray()

In [441]:
X.shape

(1500, 3715)

In [442]:
y = (target_sample['fire_rate_after'] > target_sample['fire_rate_before']).astype(int)

In [443]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [444]:
model = LogisticRegression(penalty = 'l2')
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [445]:
roc_auc_score(y_train, model.predict_proba(X_train)[:,1])

0.9414246069666068

In [446]:
roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) #not a bad start

0.7729976580796254

In [93]:
fpr, tpr, thresh = roc_curve(y_valid, model.predict_proba(X_valid)[:,1])

NameError: name 'y_valid' is not defined

In [466]:
cv_score = cross_val_score(model, X, y, cv = 5, scoring = make_scorer(roc_auc_score))

array([0.60511599, 0.59919833, 0.60086725, 0.60165292, 0.60248738])

In [33]:
cross_val_score(model, X, y, cv = 5, scoring = make_scorer(roc_auc_score))

array([0.63048189, 0.63003767, 0.62145183, 0.62491272, 0.62427039])

In [294]:
y_valid.shape

(1875,)

In [335]:
from sklearn.metrics import explained_variance_score, r2_score

In [336]:
r2_score([1,.5,0], [0,.5,1])

-3.0

# Testing: let's add some numerical features

In [607]:
sample['has_page_count'] = (~sample['no of pages'].isnull()).astype(int)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [618]:
n = X.shape[0]
split_ratio = .75
train_idx = np.random.choice(range(n), int(n*split_ratio), replace = False)
train_mask = np.full(n, fill_value = False)
for idx in train_idx:
    train_mask[idx] = True
train = train_mask
test = ~train_mask

In [596]:
sample.columns

Index(['15_day_hold?', 'assessor_stories', 'avs_street_name', 'avs_street_sfx',
       'block', 'city', 'company_name', 'contact_name', 'contact_phone',
       'contractorphone', 'description', 'estimated_cost', 'existing_stories',
       'existing_units', 'existing_use', 'expiration_date', 'file_date',
       'no of pages', 'plansets', 'proposed_stories', 'proposed_units',
       'proposed_use', 'revised_cost', 'state', 'status', 'status_date',
       'voluntary soft-story retrofit', 'has_existing_stories',
       'has_company_name', 'has_plansets', 'has_estimated_cost'],
      dtype='object')

In [619]:
numerical_X = sample[['existing_stories', 'has_existing_stories', 'has_plansets', 'has_page_count', 'no of pages']].fillna(0).values

In [620]:
X_mixed = np.concatenate((numerical_X, X), axis = 1)

In [629]:
.25*1500

375.0

In [621]:
model = LogisticRegression(penalty = 'l2')

In [622]:
model.fit(X[train], y[train])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [623]:
roc_auc_score(y[test], model.predict_proba(X[test])[:,1])

0.8154388714733543

In [624]:
X_text = model.predict_proba(X)[:,1]

In [625]:
X_final = np.concatenate((numerical_X, X_text.reshape(-1,1)),axis = 1)

In [626]:
model_b = LogisticRegression(penalty = 'l2')
model_b.fit(X_final[train], y[train])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [632]:
roc_auc_score(y[test], model_b.predict_proba(X_final[test])[:,1])

0.9131773399014779

In [630]:
f1_score(y[test], model_b.predict(X_final[test]))

0.6476868327402135

In [593]:
model_b.coef_

array([[0.08661541, 0.4135782 , 1.02716718, 6.60258556]])